In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error,mean_absolute_error
from tensorflow.keras.utils import plot_model
from tensorflow import constant_initializer
import matplotlib.pyplot as plt
import json
import shutil
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [2]:
def _exponential_initializer(min, max, dtype=None):
    def in_func(shape, dtype=dtype):
        initializer = tf.random_uniform_initializer(
                        tf.math.log(1.0),
                        tf.math.log(100.0)
                        )
        return tf.math.exp(initializer(shape))
    return in_func

class PhasedLSTM(tf.keras.layers.Layer):
    def __init__(self,
                 units,
                 leak_rate=0.001,
                 ratio_on=0.1,
                 period_init_min=0.0,
                 period_init_max=1000.0,
                 rec_activation = tf.math.sigmoid,
                 out_activation = tf.math.tanh,
                 name='plstm',
                 **kwargs):
        super(PhasedLSTM, self).__init__(name=name)
        
        self.state_size = [units,units] #This change
        self.output_size = units        #This change
        
        self.units = units
        self._leak = leak_rate
        self._ratio_on = ratio_on
        self._rec_activation = rec_activation
        self._out_activation = out_activation
        self.period_init_min = period_init_min
        self.period_init_max = period_init_max
        
        self.cell = tf.keras.layers.LSTMCell(units, **kwargs)

    def _get_cycle_ratio(self, time, phase, period):
        """Compute the cycle ratio in the dtype of the time."""
        phase_casted = tf.cast(phase, dtype=time.dtype)
        period_casted = tf.cast(period, dtype=time.dtype)
        time = tf.reshape(time, [tf.shape(time)[0],1]) #This change
        shifted_time = time - phase_casted
        cycle_ratio = (shifted_time%period_casted) / period_casted
        return tf.cast(cycle_ratio, dtype=tf.float32)        
        
    def build(self, input_shape):
        self.period = self.add_weight(
                        name="period",
                        shape=[self.units],
                        initializer=_exponential_initializer(
                                            self.period_init_min,
                                            self.period_init_max),
                        trainable=True)

        self.phase = self.add_weight(name="phase",
                                     shape=[self.units],
                                     initializer=tf.random_uniform_initializer(
                                                         0.0,
                                                         self.period),
                                     trainable=True)
        self.ratio_on = self.add_weight(name="ratio_on",
                                        shape=[self.units],
                                        initializer=constant_initializer(self._ratio_on),
                                        trainable=True)

    def call(self, input, states):
        inputs, times = input, input[:,0] #This change

        # =================================
        # CANDIDATE CELL AND HIDDEN STATE
        # =================================
        prev_hs, prev_cs = states
        output, (hs, cs) = self.cell(inputs, states)

        # =================================
        # TIME GATE
        # =================================
        cycle_ratio = self._get_cycle_ratio(times, self.phase, self.period)

        k_up = 2 * cycle_ratio / self.ratio_on
        k_down = 2 - k_up
        k_closed = self._leak * cycle_ratio

        k = tf.where(cycle_ratio < self.ratio_on, k_down, k_closed)
        k = tf.where(cycle_ratio < 0.5 * self.ratio_on, k_up, k)

        # =================================
        # UPDATE STATE USING TIME GATE VALUES
        # =================================
        new_h = k * hs + (1 - k) * prev_hs
        new_c = k * cs + (1 - k) * prev_cs

        return new_h, (new_h, new_c)

    
    
class PhasedSNForecastModel(tf.keras.Model):
    def __init__(self, units, out_steps, features, dropout=0.5):
        super().__init__()
        self.out_steps = out_steps
        self.units = units
        self.features = features
        self.dropout = dropout
        self.concat = tf.keras.layers.Concatenate()
        self.mask = tf.keras.layers.Masking(mask_value=-1.0)    
        self._init_dense()
        self._init_recurrent()



    def fowardpass(self, cells, states, denses, x, training=None):
        for i,cell in enumerate(cells):
            x, states[i] = cell(x, states=states[i],
                      training=training)
        for layer in denses:
            x = layer(x)

        return x, states

    def _warmup(self,rnn, denses, inputs):
        x, *state = rnn(inputs)
        for layer in denses:
            x = layer(x)
        return x, state

    def warmups(self,inputs):
#         prediction = self.rnn_batch_norm(inputs)
        prediction,states = self._warmup(self.rnn,self.denses,inputs)
        return prediction,states

    def _init_recurrent(self):
        cell1 = PhasedLSTM(self.units, dropout=self.dropout)
        cell2 = tf.keras.layers.LSTMCell(self.units//2, dropout=self.dropout)
        self.cells = [cell1]
        self.rnn = tf.keras.layers.RNN(self.cells, return_state=True)

    def _init_dense(self):
        dense1 = tf.keras.layers.Dense(self.units//4, activation="tanh")
        dense2 = tf.keras.layers.Dense(self.units//8, activation="tanh")
        dense3 = tf.keras.layers.Dense(self.units//16, activation="tanh")

        out = tf.keras.layers.Dense(self.features, activation="linear")
        self.denses = [
                dense1,
                tf.keras.layers.Dropout(self.dropout),
                dense2,
                tf.keras.layers.Dropout(self.dropout),
                dense3, 
                tf.keras.layers.Dropout(self.dropout), 
                out]

    def call(self, inputs, training=None):
        inputs = self.mask(inputs)
        #Creating empty tensors for predictions
        predictions = []
        prediction, states = self.warmups(inputs)

        #Saving first predictions
        predictions.append(prediction)

        for n in range(1, self.out_steps):
            prediction, states = self.fowardpass(self.cells, states, self.denses, prediction, training)
            predictions.append(prediction)

        #Stacking predictions
        predictions = tf.stack(predictions)
        predictions = tf.transpose(predictions, [1, 0, 2])
        return predictions
    

In [3]:
out_steps = 10

In [4]:
model = PhasedSNForecastModel(units=150, out_steps=out_steps,features = 3)
losses = tf.keras.losses.MeanAbsoluteError(reduction=tf.keras.losses.Reduction.NONE)
model.compile(optimizer="rmsprop", loss=losses)

In [5]:
def normalize(data):
    masked_data = np.ma.masked_where(data < 0, data)
    min_val = masked_data.min(axis=1)
    max_val = masked_data.max(axis=1)
    
    for i in range(masked_data.shape[1]):
        masked_data.data[:,i,:] = (masked_data.data[:,i,:] - min_val)/(max_val-min_val)
    
    return_data = masked_data.data
    return_data[masked_data.mask] = -1
    return return_data, min_val, max_val
    
def denormalize(data, min_val, max_val):
    masked_data = np.ma.masked_where(data < 0, data)
    
    for i in range(masked_data.shape[1]):
        masked_data.data[:,i,:] = (masked_data.data[:,i,:] * (max_val-min_val))  +  min_val
    
    return_data = masked_data.data
    return_data[masked_data.mask] = -1
    return return_data

In [6]:
data = np.load("../data/padded_x_train.npy")
len_data = data.shape[1]
data, data_min_val, data_max_val = normalize(data)
X_train, y_train = data[:,:-out_steps,:],  data[:,-out_steps:,:]

In [7]:
data_val = np.load("../data/padded_x_val.npy")
len_data = data_val.shape[1]
data_val, data_val_min_val, data_val_max_val = normalize(data_val)
X_val, y_val = data_val[:,:-out_steps,:],  data_val[:,-out_steps:,:]

In [8]:
inputs = X_train
outputs = y_train
inputs_val = X_val
outputs_val = y_val

In [9]:
#Early stops
early_stop = tf.keras.callbacks.EarlyStopping( monitor='val_loss', min_delta=1e-10, patience=10)

#Tensorboard
tensorboard = tf.keras.callbacks.TensorBoard("../data/training/logs")
shutil.rmtree("../data/training/logs",ignore_errors=True)
#Checkpoint
checkpoint = tf.keras.callbacks.ModelCheckpoint("../data/training/model_checkpoints/checkpoint", monitor='val_loss', verbose=0, save_best_only=True)

callbacks = [tensorboard,checkpoint, early_stop] # mag_early_stop,fid_early_stop,dt_early_stop,

In [10]:
MAX_EPOCHS=1000
history = model.fit(inputs,outputs,
                    batch_size=300, 
                    epochs=MAX_EPOCHS, 
                    validation_data=(inputs_val,outputs_val), 
                    callbacks=callbacks)

Epoch 1/1000
1/9 [==>...........................] - ETA: 0s - loss: 0.5240WARNING:tensorflow:From /home/camilo/anaconda3/lib/python3.8/site-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
9/9 [==============================] - 4s 439ms/step - loss: 0.4198 - val_loss: 0.3190
Epoch 2/1000
9/9 [==============================] - 3s 338ms/step - loss: 0.3532 - val_loss: 0.2863
Epoch 3/1000
9/9 [==============================] - 4s 428ms/step - loss: 0.3341 - val_loss: 0.2675
Epoch 4/1000
9/9 [==============================] - 4s 419ms/step - loss: 0.3210 - val_loss: 0.2471
Epoch 5/1000
9/9 [==============================] - 4s 410ms/step - loss: 0.3072 - val_loss: 0.2685
Epoch 6/1000
9/9 [==============================] - 4s 409ms/step - loss: 0.2965 - val_loss: 0.2264
Epoch 7/1000
9/9 [==============================]

In [3]:
history_dict = history.history
json.dump(history_dict, open("../data/training/history_model.json", 'w'))

NameError: name 'history' is not defined

In [4]:
model.save_weights("../data/sn_model.h5")

NameError: name 'model' is not defined